## Event reproducibility in RSC
### using data from Franco and Goard 2021

In [53]:
from platform import python_version
print(python_version())

%run -i 'imports_functions.py' 

%matplotlib inline

3.10.4


### Data preprocessing

The original file `twoPhotonData.mat` is available on [Zenodo](https://zenodo.org/record/4734407/files/twoPhotonData.mat).



In [54]:
if os.path.exists('goard/spiketrains_twoPhotonData_unselected.npy'):
    all_spiketrains = np.load('goard/spiketrains_twoPhotonData_unselected.npy', allow_pickle=True)
    time = np.load('goard/time_twoPhotonData_unselected.npy', allow_pickle=True) # all time samples
    print("... loaded",len(all_spiketrains), "spiketrains for", len(time), "intervals")

exp_path = os.getcwd() + "/goard/"
print(exp_path)
frame_duration = 0.1 # sec, 10 frames per second, see Methods in Franco and Goard 2021

... loaded 7770 spiketrains for 20160 intervals
/home/jovyan/work/goard/


We should separate the 29 sessions


In [55]:
cellsPerSession = [
    279, 136, 317, 296, 292, 281, 258, 279, 279, 271, 
    177, 257, 296, 253, 329, 230, 292, 181, 201, 292, 
    241, 382, 239, 249, 315, 262, 237, 321, 328
]
indices_cellsPerSession = np.cumsum(cellsPerSession)
print(indices_cellsPerSession)

sessions = np.split(all_spiketrains, indices_cellsPerSession)[:-1] # omit last empty split


[ 279  415  732 1028 1320 1601 1859 2138 2417 2688 2865 3122 3418 3671
 4000 4230 4522 4703 4904 5196 5437 5819 6058 6307 6622 6884 7121 7442
 7770]


## Dynamical analysis

Analysis for each session.

On average, the longest responses were for the 4th trial. We take it.

In [78]:
global_events = [[] for i in range(29)] # by session
global_events_vector = [[] for i in range(29)] # by session
global_events_sec = []
global_events_duration = []
global_cluster_number = []
global_cluster_selfsimilarity = [[] for i in range(29)]

core_reproducibility_perc = 60 # threshold for detecting cores

for scan_id,scan_spiketrains in enumerate(sessions):
    print(len(scan_spiketrains))
    # we only take trial 4, starting at 1500 sec
    scan_spiketrains = [ np.nan_to_num(st[np.argwhere(np.array(st)>1500)[0][0]:], nan=0., posinf=0., neginf=0.).tolist() for st in scan_spiketrains]
    
    start_time = 1500
    exp_tstart = start_time
    stop_time = max([max(st) if len(st) else 0 for st in scan_spiketrains])
    
    print("... producing spike rasterplot per session")
    fig = plt.figure()
    for row,train in enumerate(scan_spiketrains):
        print(any(v==np.nan for v in train))
        plt.scatter( train, [row]*len(train), marker='o', edgecolors='none', s=1, c='k' )
    plt.ylabel("cell IDs")
    plt.xlabel("time (s)")
    fig.savefig(exp_path+'/results/rasterplot_twoPhotonData_scan%s.png'%scan_id, transparent=True, dpi=800)
    plt.close()
    fig.clear()
    fig.clf()
    
    extra_toBremoved = [21]

    ophys_cell_ids = list(range(len(scan_spiketrains)))
    ophys_cell_indexes = ophys_cell_ids # alias

    %run "dynamical_analysis.ipynb"

    global_events[scan_id].extend(events)
    global_events_vector[scan_id].extend(events_vectors)
    global_events_sec.append(events_sec)
    global_events_duration.extend(events_durations_f)
    global_cluster_number.append(nclusters)
    global_cluster_selfsimilarity.extend(reproducibility_list)


279
... producing spike rasterplot per session
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False

ValueError: The condensed distance matrix must contain only finite values.

ValueError: The condensed distance matrix must contain only finite values.